# ANOVOS - Data Transformer
Following notebook shows the list of functions related to "data transformer" module provided under ANOVOS package and how it can be invoked accordingly.
- [Attribute Binning](#Attribute-Binning)
- [Monotonic Binning](#Monotonic-Binning)
- [Categorical Attribute to Numerical Attribute Conversion](#Categorical-Attribute-to-Numerical-Attribute-Conversion)
    - [Categorical to Numerical - Unsupervised](#Categorical-to-Numerical---Unsupervised)
    - [Categorical to Numerical - Supervised](#Categorical-to-Numerical---Supervised)
- [Attribute Rescaling](#Attribute-Rescaling)
    - [Z Standardization](#Z-Standardization)
    - [IQR Standardization](#IQR-Standardization)
    - [Normalization](#Normalization)
- [Missing Value Imputation](#Missing-Value-Imputation)
    - [Imputation MMM](#Imputation-MMM)
    - [Imputation Sklearn](#Imputation-Sklearn)
    - [Imputation Matrix Factorization](#Imputation-Matrix-Factorization)
    - [Auto Imputation](#Auto-Imputation)
- [Latent Features Generation](#Latent-Features-Generation)
    - [Autoencoder Latent Features](#Autoencoder-Latent-Features)
    - [PCA Latent Features](#PCA-Latent-Features)
- [Feature Transformation](#Feature-Transformation)
- [Box Cox Transformation](#Box-Cox-Transformation)
- [Outlier Categories Treatment](#Outlier-Categories-Treatment)
- [Expression Parser](#Expression-Parser)

**Setting Spark Session**

In [2]:
from anovos.shared.spark import *

sc.setLogLevel("ERROR")
import warnings
warnings.filterwarnings('ignore')

**Input/Output Path** 

In [3]:
inputPath = "../data/income_dataset/csv"
outputPath = "../output/income_dataset/data_transformer"

**Read Input Data** 

In [4]:
from anovos.data_ingest.data_ingest import read_dataset
from pyspark.sql import functions as F
df = read_dataset(spark, file_path = inputPath, file_type = "csv",
                  file_configs = {"header": "True", "delimiter": "," , "inferSchema": "True"})
df = df.drop("dt_1", "dt_2")
df.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,UnitedStates,<=50K
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,UnitedStates,<=50K
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,UnitedStates,<=50K
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,UnitedStates,<=50K
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


# Attribute Binning
- API specification of function **attribute_binning** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only
- 2 binning options: Equal Range Binning (each bin is of equal size/width) and Equal Frequency Binning (each bin has equal no. of rows)

In [4]:
from anovos.data_transformer.transformers import attribute_binning

In [5]:
# Example 1 - Equal range binning + append transformed columns at the end
odf = attribute_binning(spark, idf=df, list_of_cols=["education-num", "hours-per-week"], method_type="equal_range", 
                        bin_size=5, output_mode="append", print_impact=True)

odf.toPandas().head(5)

+---------------------+-------------+
|attribute            |unique_values|
+---------------------+-------------+
|education-num_binned |5            |
|hours-per-week_binned|5            |
+---------------------+-------------+



,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,education-num_binned,hours-per-week_binned
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,UnitedStates,<=50K,4.0,3.0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,UnitedStates,<=50K,4.0,1.0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,UnitedStates,<=50K,3.0,3.0
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,UnitedStates,<=50K,2.0,3.0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,4.0,3.0


In [6]:
# Distinct values after binning
odf.select('hours-per-week_binned').distinct().orderBy('hours-per-week_binned').toPandas().head(10)

,hours-per-week_binned
0,NaN
1,1.0
2,2.0
3,3.0
4,4.0
5,5.0


In [7]:
# Example 2 - Equal frequency binning + replace original columns by transformed ones (default)
odf = attribute_binning(spark, df, list_of_cols=["education-num", "hours-per-week"], method_type="equal_frequency", 
                        bin_size=5, print_impact=True)

odf.toPandas().head(5)

+--------------+-------------+
|attribute     |unique_values|
+--------------+-------------+
|education-num |4            |
|hours-per-week|4            |
+--------------+-------------+



,ifa,age,workclass,fnlwgt,logfnl,empty,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,native-country,income,education-num,hours-per-week
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,UnitedStates,<=50K,4.0,2.0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,UnitedStates,<=50K,4.0,1.0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,UnitedStates,<=50K,1.0,2.0
3,4a,53.0,Private,234721.0,5.370552,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,UnitedStates,<=50K,1.0,2.0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,Cuba,<=50K,4.0,2.0


In [8]:
# Distinct values after binning
odf.select('hours-per-week').distinct().orderBy('hours-per-week').toPandas().head(10)

,hours-per-week
0,NaN
1,1.0
2,2.0
3,4.0
4,5.0


In [9]:
# Example 3 - Equal frequency binning + save binning model
odf = attribute_binning(spark, df, list_of_cols=["education-num", "hours-per-week"], method_type="equal_frequency", 
                        bin_size=5, pre_existing_model=False, model_path=outputPath + "/attribute_binning")

odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,native-country,income,education-num,hours-per-week
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,UnitedStates,<=50K,4.0,2.0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,UnitedStates,<=50K,4.0,1.0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,UnitedStates,<=50K,1.0,2.0
3,4a,53.0,Private,234721.0,5.370552,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,UnitedStates,<=50K,1.0,2.0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,Cuba,<=50K,4.0,2.0


In [10]:
# Example 4 - Equal frequency binning + use pre-saved model
odf = attribute_binning(spark, df, list_of_cols=["education-num", "hours-per-week"], 
                        pre_existing_model=True, model_path=outputPath + "/attribute_binning")
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,native-country,income,education-num,hours-per-week
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,UnitedStates,<=50K,4.0,2.0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,UnitedStates,<=50K,4.0,1.0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,UnitedStates,<=50K,1.0,2.0
3,4a,53.0,Private,234721.0,5.370552,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,UnitedStates,<=50K,1.0,2.0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,Cuba,<=50K,4.0,2.0


# Monotonic Binning
- API specification of function **monotonic_binning** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Bin size is computed dynamically

In [11]:
from anovos.data_transformer.transformers import monotonic_binning

In [12]:
# Example 1 - Equal Range Binning + append tranformed columns at the end
odf = monotonic_binning(spark, df, list_of_cols=["education-num", "hours-per-week"], label_col="income", 
                        event_label=">50K", bin_method="equal_range", output_mode="append")
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,education-num_binned,hours-per-week_binned
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,UnitedStates,<=50K,6.0,2.0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,UnitedStates,<=50K,6.0,1.0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,UnitedStates,<=50K,4.0,2.0
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,UnitedStates,<=50K,3.0,2.0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,6.0,2.0


In [13]:
# Distinct values for hours-per-week after binning 
odf.select("hours-per-week_binned").distinct().orderBy('hours-per-week_binned').toPandas()

,hours-per-week_binned
0,NaN
1,1.0
2,2.0
3,3.0


In [14]:
# Example 2 - Equal Frequency Binning + replace original columns by transformed ones (default)
odf = monotonic_binning(spark, df, list_of_cols=["education-num", "hours-per-week"], label_col="income", 
                        event_label=">50K", bin_method="equal_frequency")
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,native-country,income,education-num,hours-per-week
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,UnitedStates,<=50K,12.0,2.0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,UnitedStates,<=50K,12.0,1.0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,UnitedStates,<=50K,3.0,2.0
3,4a,53.0,Private,234721.0,5.370552,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,UnitedStates,<=50K,2.0,2.0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,Cuba,<=50K,12.0,2.0


In [15]:
# Distinct values for hours-per-week after binning
odf.select("hours-per-week").distinct().orderBy('hours-per-week').toPandas()

,hours-per-week
0,NaN
1,1.0
2,2.0
3,6.0
4,7.0


# Categorical Attribute to Numerical Attribute Conversion

## Categorical to Numerical - Unsupervised
- API specification of function **cat_to_num_unsupervised** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports Label Encoding (default) and One hot encoding

In [5]:
from anovos.data_transformer.transformers import cat_to_num_unsupervised

In [6]:
# Example 1 - with mandatory arguments (Label Encoding)
odf = cat_to_num_unsupervised(spark, df)
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,15092,NaN,10.0,77516.0,4.889391,None,2.0,13.0,1.0,3.0,1.0,0.0,0.0,2174.0,0.0,40.0,42,0
1,22895,NaN,1.0,83311.0,4.920702,None,2.0,13.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,13.0,42,0
2,28742,38.0,0.0,215646.0,5.333741,None,0.0,9.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,40.0,42,0
3,4124,53.0,0.0,234721.0,5.370552,None,5.0,7.0,0.0,9.0,0.0,1.0,0.0,0.0,0.0,40.0,42,0
4,11005,NaN,0.0,338409.0,5.529442,None,2.0,13.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,40.0,10,0


In [7]:
# Example 2 - 'all' columns (excluding drop_cols) + print impact
odf = cat_to_num_unsupervised(spark, df, list_of_cols='all', drop_cols=['ifa'], print_impact=True)
odf.toPandas().head(5)

Before


+-------+------+-----+-----------+-------+-----------+-----+------------+-------------+--------------+----------------+------------+-------+-----+------------+------------+--------------+--------------+------+
|summary|ifa   |age  |workclass  |fnlwgt |logfnl     |empty|education   |education-num|marital-status|occupation      |relationship|race   |sex  |capital-gain|capital-loss|hours-per-week|native-country|income|
+-------+------+-----+-----------+-------+-----------+-----+------------+-------------+--------------+----------------+------------+-------+-----+------------+------------+--------------+--------------+------+
|count  |32561 |32500|32558      |32546  |12168      |0    |32040       |32530        |32135         |32549           |32557       |32247  |32557|32548       |32549       |32452         |32561         |32561 |
|min    |10000a|17   | Private   |12285  |4.283617786|null |10th        |1            |?             |?               |*           |*      |?    |0           |0

+-------+------+-----+---------+-------+-----------+-----+---------+-------------+--------------+----------+------------+-----+-----+------------+------------+--------------+--------------+------+
|summary|ifa   |age  |workclass|fnlwgt |logfnl     |empty|education|education-num|marital-status|occupation|relationship|race |sex  |capital-gain|capital-loss|hours-per-week|native-country|income|
+-------+------+-----+---------+-------+-----------+-----+---------+-------------+--------------+----------+------------+-----+-----+------------+------------+--------------+--------------+------+
|count  |32561 |32500|32558    |32546  |12168      |0    |32040    |32530        |32135         |32549     |32557       |32247|32557|32548       |32549       |32452         |32561         |32561 |
|min    |10000a|17   |0        |12285  |4.283617786|null |0        |1            |0             |0         |0           |0    |0    |0           |0           |1             |0             |0     |
|max    |9a    

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,1a,NaN,10.0,77516.0,4.889391,None,2.0,13.0,1.0,3.0,1.0,0.0,0.0,2174.0,0.0,40.0,42,0
1,2a,NaN,1.0,83311.0,4.920702,None,2.0,13.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,13.0,42,0
2,3a,38.0,0.0,215646.0,5.333741,None,0.0,9.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,40.0,42,0
3,4a,53.0,0.0,234721.0,5.370552,None,5.0,7.0,0.0,9.0,0.0,1.0,0.0,0.0,0.0,40.0,42,0
4,5a,NaN,0.0,338409.0,5.529442,None,2.0,13.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,40.0,10,0


In [19]:
# Example 3 - selected categorical columns + assign unique integers based on alphabetical order (asc)
odf = cat_to_num_unsupervised(spark, df, list_of_cols='all', drop_cols=['ifa'], index_order='alphabetAsc')
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,1a,NaN,1.0,77516.0,4.889391,None,9.0,13.0,4.0,1.0,3.0,6.0,2.0,2174.0,0.0,40.0,41,0
1,2a,NaN,8.0,83311.0,4.920702,None,9.0,13.0,3.0,4.0,2.0,6.0,2.0,0.0,0.0,13.0,41,0
2,3a,38.0,6.0,215646.0,5.333741,None,11.0,9.0,1.0,6.0,3.0,6.0,2.0,0.0,0.0,40.0,41,0
3,4a,53.0,6.0,234721.0,5.370552,None,1.0,7.0,3.0,6.0,2.0,4.0,2.0,0.0,0.0,40.0,41,0
4,5a,NaN,6.0,338409.0,5.529442,None,9.0,13.0,3.0,10.0,7.0,4.0,1.0,0.0,0.0,40.0,6,0


In [13]:
# Example 4 - selected categorical columns + one hot encoding + print impact
odf = cat_to_num_unsupervised(spark, df, list_of_cols=['race', 'sex'], 
                              method_type="onehot_encoding", print_impact=True)
odf.toPandas().head(5)

Before
root
 |-- ifa: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- logfnl: double (nullable = true)
 |-- empty: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)

After
root
 |-- ifa: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- logfnl: double (nullable = true)
 |-- empty: string (nullable = true)
 |-- educatio

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,...,race_4,race_5,race_6,race_7,race_8,race_9,sex_0,sex_1,sex_2,sex_3
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,...,0,0,0,0,0,0,1,0,0,0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,...,0,0,0,0,0,0,1,0,0,0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,...,0,0,0,0,0,0,1,0,0,0
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,...,0,0,0,0,0,0,1,0,0,0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,...,0,0,0,0,0,0,0,1,0,0


In [14]:
# Example 5 - one hot encoding + save model
odf = cat_to_num_unsupervised(spark, df, list_of_cols=['race', 'sex'], method_type="onehot_encoding", 
                              pre_existing_model=False, model_path=outputPath)
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,...,race_4,race_5,race_6,race_7,race_8,race_9,sex_0,sex_1,sex_2,sex_3
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,...,0,0,0,0,0,0,1,0,0,0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,...,0,0,0,0,0,0,1,0,0,0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,...,0,0,0,0,0,0,1,0,0,0
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,...,0,0,0,0,0,0,1,0,0,0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,...,0,0,0,0,0,0,0,1,0,0


In [15]:
# Example 6 - one hot encoding + use pre-saved model
odf = cat_to_num_unsupervised(spark, df, list_of_cols=['race', 'sex'], method_type="onehot_encoding", 
                              pre_existing_model=True, model_path=outputPath)
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,...,race_4,race_5,race_6,race_7,race_8,race_9,sex_0,sex_1,sex_2,sex_3
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,...,0,0,0,0,0,0,1,0,0,0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,...,0,0,0,0,0,0,1,0,0,0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,...,0,0,0,0,0,0,1,0,0,0
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,...,0,0,0,0,0,0,1,0,0,0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,...,0,0,0,0,0,0,0,1,0,0


## Categorical to Numerical - Supervised
- API specification of function **cat_to_num_supervised**  can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>

In [23]:
from anovos.data_transformer.transformers import cat_to_num_supervised

In [24]:
# Example 1 - 'all' columns (excluding drop_cols) + print impact 
odf = cat_to_num_supervised(spark, idf=df, list_of_cols="all", drop_cols="ifa", 
                            label_col="income", event_label=">50K", print_impact=True)

Before: 


+-------+--------------+------------+-----+-----------+-----+--------------+----------------+-------+------------+
|summary|marital-status|education   |sex  |workclass  |empty|native-country|occupation      |race   |relationship|
+-------+--------------+------------+-----+-----------+-----+--------------+----------------+-------+------------+
|count  |32135         |32040       |32557|32558      |0    |32561         |32549           |32247  |32557       |
|min    |?             |10th        |?    | Private   |null |*             |?               |*      |*           |
|max    |Widowed       |Some-college|Male |Without-pay|null |Yugoslavia    |Transport-moving|Whitess|Wife        |
+-------+--------------+------------+-----+-----------+-----+--------------+----------------+-------+------------+

After: 
+-------+--------------+---------+------+---------+------+--------------+----------+-----+------------+
|summary|marital-status|education|sex   |workclass|empty |native-country|occupatio

In [25]:
# Example 2 - selected cateogrical columns + append generated columns + print impact
odf = cat_to_num_supervised(spark, idf=df, list_of_cols=['relationship', 'marital-status'],
                            label_col="income", event_label=">50K", output_mode="append", print_impact=True)

Before: 
+-------+------------+--------------+
|summary|relationship|marital-status|
+-------+------------+--------------+
|count  |32557       |32135         |
|min    |*           |?             |
|max    |Wife        |Widowed       |
+-------+------------+--------------+

After: 
+-------+--------------------+----------------------+
|summary|relationship_encoded|marital-status_encoded|
+-------+--------------------+----------------------+
|count  |32557               |32135                 |
|min    |0.0                 |0.0458                |
|max    |0.4748              |0.4471                |
+-------+--------------------+----------------------+



In [26]:
# Example 3 - selected categorical columns + append generated column + save model
odf = cat_to_num_supervised(spark, idf=df, list_of_cols=['relationship', 'marital-status', 'workclass'], 
                            label_col="income", event_label=">50K", model_path=outputPath, output_mode="append")

In [27]:
# Example 4 - selected categorical columns + use pre-saved model
odf = cat_to_num_supervised(spark, idf=df, list_of_cols=['relationship', 'marital-status'], 
                            label_col="income", event_label=">50K", pre_existing_model=True, 
                            model_path=outputPath, print_impact=True)

Before: 
+-------+------------+--------------+
|summary|relationship|marital-status|
+-------+------------+--------------+
|count  |32557       |32135         |
|min    |*           |?             |
|max    |Wife        |Widowed       |
+-------+------------+--------------+

After: 
+-------+------------+--------------+
|summary|relationship|marital-status|
+-------+------------+--------------+
|count  |32557       |32135         |
|min    |0.0         |0.0458        |
|max    |0.4748      |0.4471        |
+-------+------------+--------------+



# Attribute Rescaling

## Z Standardization
- API specification of function **z_standardization** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only

In [28]:
from anovos.data_transformer.transformers import z_standardization

In [29]:
# Example 1 - with mandatory arguments
odf = z_standardization(spark, idf=df)

In [30]:
# Example 2 - selected columns + print impact
odf = z_standardization(spark, idf=df, list_of_cols=['fnlwgt', 'age', 'hours-per-week'], print_impact=True)

Before: 
+-------+------------------+------------------+------------------+
|summary|hours-per-week    |age               |fnlwgt            |
+-------+------------------+------------------+------------------+
|count  |32452             |32500             |32546             |
|mean   |40.24972266732405 |38.506492307692305|189781.83180728814|
|stddev |11.914337669272212|13.508497735339255|105563.06445057027|
|min    |1                 |17                |12285             |
|max    |94                |85                |1484705           |
+-------+------------------+------------------+------------------+

After: 
+-------+-----------------------+----------------------+---------------------+
|summary|hours-per-week         |age                   |fnlwgt               |
+-------+-----------------------+----------------------+---------------------+
|count  |32452                  |32500                 |32546                |
|mean   |-1.2156752033466475E-15|1.4483798775717543E-16|9.16157

In [31]:
# Example 3 - 'all' columns + save model + print impact
odf = z_standardization(spark, idf=df, list_of_cols='all', model_path=outputPath)

In [32]:
# Example 4 - selected columns + append new columns + use pre-saved model + print impact
odf = z_standardization(spark, idf=df, list_of_cols=['fnlwgt', 'age', 'hours-per-week'], 
                        pre_existing_model=True, model_path=outputPath, output_mode='append', print_impact=True)

Before: 
+-------+------------------+------------------+------------------+
|summary|hours-per-week    |age               |fnlwgt            |
+-------+------------------+------------------+------------------+
|count  |32452             |32500             |32546             |
|mean   |40.24972266732405 |38.506492307692305|189781.83180728814|
|stddev |11.914337669272212|13.508497735339255|105563.06445057027|
|min    |1                 |17                |12285             |
|max    |94                |85                |1484705           |
+-------+------------------+------------------+------------------+

After: 
+-------+-----------------------+----------------------+---------------------+
|summary|hours-per-week_scaled  |age_scaled            |fnlwgt_scaled        |
+-------+-----------------------+----------------------+---------------------+
|count  |32452                  |32500                 |32546                |
|mean   |-1.2156752033466475E-15|1.4483798775717543E-16|9.16157

## IQR Standardization
- API specification of function **IQR_standardization** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only

In [33]:
from anovos.data_transformer.transformers import IQR_standardization

In [34]:
# Example 1 - with mandatory arguments
odf = IQR_standardization(spark, idf=df)

/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/anovos/data_transformer/transformers.py:1085: UserWarning: The following column(s) are excluded from standardization because the 75th and 25th percentiles are the same:['capital-loss', 'capital-gain']
  + str(excluded_cols)


In [35]:
# Example 2 - selected columns + print impact
odf = IQR_standardization(spark, idf=df, list_of_cols=['fnlwgt', 'age', 'hours-per-week'], print_impact=True)

Before: 
+-------+------------------+------------------+------------------+
|summary|hours-per-week    |age               |fnlwgt            |
+-------+------------------+------------------+------------------+
|count  |32452             |32500             |32546             |
|mean   |40.24972266732405 |38.506492307692305|189781.83180728814|
|stddev |11.914337669272212|13.508497735339255|105563.06445057027|
|min    |1                 |17                |12285             |
|max    |94                |85                |1484705           |
+-------+------------------+------------------+------------------+

After: 
+-------+-------------------+-------------------+-------------------+
|summary|hours-per-week     |age                |fnlwgt             |
+-------+-------------------+-------------------+-------------------+
|count  |32452              |32500              |32546              |
|mean   |0.04994453346480964|0.07928906882590958|0.10904578788801947|
|stddev |2.3828675338544496 |

In [36]:
# Example 3 - 'all' columns + save model + print impact
odf = IQR_standardization(spark, idf=df, list_of_cols='all', model_path=outputPath)

In [37]:
# Example 4 - selected columns + append new columns + use pre-saved model + print impact
odf = IQR_standardization(spark, idf=df, list_of_cols=['fnlwgt', 'age', 'hours-per-week'], 
                          pre_existing_model=True, model_path=outputPath, output_mode='append', print_impact=True)

Before: 
+-------+------------------+------------------+------------------+
|summary|hours-per-week    |age               |fnlwgt            |
+-------+------------------+------------------+------------------+
|count  |32452             |32500             |32546             |
|mean   |40.24972266732405 |38.506492307692305|189781.83180728814|
|stddev |11.914337669272212|13.508497735339255|105563.06445057027|
|min    |1                 |17                |12285             |
|max    |94                |85                |1484705           |
+-------+------------------+------------------+------------------+

After: 
+-------+---------------------+-------------------+-------------------+
|summary|hours-per-week_scaled|age_scaled         |fnlwgt_scaled      |
+-------+---------------------+-------------------+-------------------+
|count  |32452                |32500              |32546              |
|mean   |0.04994453346480964  |0.07928906882590958|0.10904578788801947|
|stddev |2.38286753

## Normalization
- API specification of function **normalization** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only

In [5]:
from anovos.data_transformer.transformers import normalization

In [6]:
# Example 1 - with mandatory arguments
odf = normalization(idf=df)

In [7]:
# Example 2 - 'all' columns + print impact
odf = normalization(idf=df, list_of_cols='all', print_impact=True)

Before: 
+-------+-------------------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|logfnl             |hours-per-week    |education-num     |capital-gain      |fnlwgt            |age               |capital-loss     |
+-------+-------------------+------------------+------------------+------------------+------------------+------------------+-----------------+
|count  |12168              |32452             |32530             |32548             |32546             |32500             |32549            |
|mean   |5.2054654851899365 |40.24972266732405 |10.080971411005226|1077.6959567408135|189781.83180728814|38.506492307692305|87.3360164674798 |
|stddev |0.27424241727170395|11.914337669272227|2.5725103263986946|7386.624857802765 |105563.06445057005|13.508497735339288|403.0310072565714|
|min    |4.283617786        |1                 |1                 |0                 |12285             |17                |0        

[Stage 7:=============================>                             (1 + 1) / 2]

+-------+-------------------+-------------------+------------------+--------------------+-------------------+-------------------+--------------------+
|summary|logfnl             |hours-per-week     |education-num     |capital-gain        |fnlwgt             |age                |capital-loss        |
+-------+-------------------+-------------------+------------------+--------------------+-------------------+-------------------+--------------------+
|count  |12168              |32452              |32530             |32548               |32546              |32500              |32549               |
|mean   |0.5106965512077881 |0.42204001986123296|0.6053981130856241|0.010777067434618917|0.12054769144314768|0.3162719473860012 |0.020049590646946842|
|stddev |0.15192819447055378|0.12811115499749567|0.1715006866238256|0.073866987430084   |0.07169358238758687|0.19865437746694342|0.09252318850237391 |
|min    |0.0                |0.0                |0.0               |0.0                 |0.0  

In [8]:
# Example 3 - selected columns + save model
odf = normalization(idf=df, list_of_cols=['fnlwgt', 'age', 'hours-per-week'], 
                    pre_existing_model=False, model_path=outputPath, print_impact=True)

Before: 
+-------+------------------+------------------+------------------+
|summary|age               |fnlwgt            |hours-per-week    |
+-------+------------------+------------------+------------------+
|count  |32500             |32546             |32452             |
|mean   |38.506492307692305|189781.83180728814|40.24972266732405 |
|stddev |13.508497735339288|105563.06445057005|11.914337669272227|
|min    |17                |12285             |1                 |
|max    |85                |1484705           |94                |
+-------+------------------+------------------+------------------+

After: 


[Stage 18:=============================>                            (1 + 1) / 2]

+-------+-------------------+-------------------+-------------------+
|summary|age                |fnlwgt             |hours-per-week     |
+-------+-------------------+-------------------+-------------------+
|count  |32500              |32546              |32452              |
|mean   |0.3162719473860012 |0.12054769144314768|0.42204001986123296|
|stddev |0.19865437746694342|0.07169358238758687|0.12811115499749567|
|min    |0.0                |0.0                |0.0                |
|max    |1.0                |1.0                |1.0                |
+-------+-------------------+-------------------+-------------------+



In [9]:
# Example 4 - selected columns + append new columns + use pre-saved model + print impact
odf = normalization(idf=df, list_of_cols=['fnlwgt', 'age', 'hours-per-week'], 
                    pre_existing_model=True, model_path=outputPath, output_mode='append', print_impact=True)

Before: 
+-------+------------------+------------------+------------------+
|summary|age               |fnlwgt            |hours-per-week    |
+-------+------------------+------------------+------------------+
|count  |32500             |32546             |32452             |
|mean   |38.506492307692305|189781.83180728814|40.24972266732405 |
|stddev |13.508497735339288|105563.06445057005|11.914337669272227|
|min    |17                |12285             |1                 |
|max    |85                |1484705           |94                |
+-------+------------------+------------------+------------------+

After: 


[Stage 27:=============================>                            (1 + 1) / 2]

+-------+-------------------+-------------------+---------------------+
|summary|age_scaled         |fnlwgt_scaled      |hours-per-week_scaled|
+-------+-------------------+-------------------+---------------------+
|count  |32500              |32546              |32452                |
|mean   |0.3162719473860012 |0.12054769144314768|0.42204001986123296  |
|stddev |0.19865437746694342|0.07169358238758687|0.12811115499749567  |
|min    |0.0                |0.0                |0.0                  |
|max    |1.0                |1.0                |1.0                  |
+-------+-------------------+-------------------+---------------------+



# Missing Value Imputation

## Imputation MMM
- API specification of function **imputation_MMM** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- 2 options for numerical  attributes: median and mean
- Mode is only option for categorical attributes

In [43]:
from anovos.data_transformer.transformers import imputation_MMM

In [44]:
# Example 1 - with mandatory arguments + print impact
odf = imputation_MMM(spark, df, print_impact=True)

+--------------+-------------------+------------------+
|attribute     |missingCount_before|missingCount_after|
+--------------+-------------------+------------------+
|education-num |31                 |0                 |
|workclass     |3                  |0                 |
|education     |521                |0                 |
|race          |314                |0                 |
|relationship  |4                  |0                 |
|capital-gain  |13                 |0                 |
|capital-loss  |12                 |0                 |
|age           |61                 |0                 |
|hours-per-week|109                |0                 |
|fnlwgt        |15                 |0                 |
|marital-status|426                |0                 |
|sex           |4                  |0                 |
|occupation    |12                 |0                 |
|logfnl        |20393              |0                 |
|empty         |32561              |0           

In [45]:
# Example 2 - use mean for numerical columns + append transformed columns at the end
odf = imputation_MMM(spark, df, list_of_cols='all', method_type="mean", output_mode="append")
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,...,logfnl_imputed,hours-per-week_imputed,education_imputed,marital-status_imputed,sex_imputed,workclass_imputed,empty_imputed,occupation_imputed,race_imputed,relationship_imputed
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,...,4.889391,40,Bachelors,Never-married,Male,State-gov,None,Adm-clerical,White,Not-in-family
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,...,4.920702,13,Bachelors,Married-civ-spouse,Male,Self-emp-not-inc,None,Exec-managerial,White,Husband
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,...,5.333741,40,HS-grad,Divorced,Male,Private,None,Handlers-cleaners,White,Not-in-family
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,...,5.370552,40,11th,Married-civ-spouse,Male,Private,None,Handlers-cleaners,Black,Husband
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,...,5.529442,40,Bachelors,Married-civ-spouse,Female,Private,None,Prof-specialty,Black,Wife


In [46]:
odf.select('education-num', 'education-num_imputed').where(F.col("education-num").isNull()).distinct().toPandas().head(5)

,education-num,education-num_imputed
0,None,10


In [47]:
# Example 3 - save model
odf = imputation_MMM(spark, df, pre_existing_model=False, model_path=outputPath)

In [48]:
# Example 4 - use pre-saved model
odf = imputation_MMM(spark, df, pre_existing_model=True, model_path=outputPath)
odf.toPandas().head(5)

,ifa,native-country,income,education-num,age,fnlwgt,capital-loss,capital-gain,logfnl,hours-per-week,education,marital-status,sex,workclass,empty,occupation,race,relationship
0,1a,UnitedStates,<=50K,13,37,77516,0,2174,4.889391,40,Bachelors,Never-married,Male,State-gov,None,Adm-clerical,White,Not-in-family
1,2a,UnitedStates,<=50K,13,37,83311,0,0,4.920702,13,Bachelors,Married-civ-spouse,Male,Self-emp-not-inc,None,Exec-managerial,White,Husband
2,3a,UnitedStates,<=50K,9,38,215646,0,0,5.333741,40,HS-grad,Divorced,Male,Private,None,Handlers-cleaners,White,Not-in-family
3,4a,UnitedStates,<=50K,7,53,234721,0,0,5.370552,40,11th,Married-civ-spouse,Male,Private,None,Handlers-cleaners,Black,Husband
4,5a,Cuba,<=50K,13,37,338409,0,0,5.529442,40,Bachelors,Married-civ-spouse,Female,Private,None,Prof-specialty,Black,Wife


In [49]:
# Example 5 - selected columns + use pre-saved stats
from anovos.data_analyzer.stats_generator import measures_of_counts, measures_of_centralTendency
from anovos.data_ingest.data_ingest import write_dataset
missing = write_dataset(measures_of_counts(spark, df),outputPath+"/missing","parquet", file_configs={"mode":"overwrite"})
mode = write_dataset(measures_of_centralTendency(spark, df),outputPath+"/mode","parquet", file_configs={"mode":"overwrite"})

odf = imputation_MMM(spark, df, list_of_cols=['marital-status', 'sex', 'occupation', 'age'], 
                     stats_missing={"file_path":outputPath+"/missing", "file_type": "parquet"}, 
                     stats_mode={"file_path":outputPath+"/mode", "file_type": "parquet"}, print_impact=True)
odf.toPandas().head(5)

+--------------+-------------------+------------------+
|attribute     |missingCount_before|missingCount_after|
+--------------+-------------------+------------------+
|age           |61                 |0                 |
|marital-status|426                |0                 |
|occupation    |12                 |0                 |
|sex           |4                  |0                 |
+--------------+-------------------+------------------+



,ifa,workclass,fnlwgt,logfnl,empty,education,education-num,relationship,race,capital-gain,capital-loss,hours-per-week,native-country,income,age,marital-status,sex,occupation
0,1a,State-gov,77516.0,4.889391,None,Bachelors,13.0,Not-in-family,White,2174.0,0.0,40.0,UnitedStates,<=50K,37,Never-married,Male,Adm-clerical
1,2a,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Husband,White,0.0,0.0,13.0,UnitedStates,<=50K,37,Married-civ-spouse,Male,Exec-managerial
2,3a,Private,215646.0,5.333741,None,HS-grad,9.0,Not-in-family,White,0.0,0.0,40.0,UnitedStates,<=50K,38,Divorced,Male,Handlers-cleaners
3,4a,Private,234721.0,5.370552,None,11th,7.0,Husband,Black,0.0,0.0,40.0,UnitedStates,<=50K,53,Married-civ-spouse,Male,Handlers-cleaners
4,5a,Private,338409.0,5.529442,None,Bachelors,13.0,Wife,Black,0.0,0.0,40.0,Cuba,<=50K,37,Married-civ-spouse,Female,Prof-specialty


## Imputation Sklearn
- API specification of function **imputation_sklearn** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only
- 2 options supported: KNN and regression

In [10]:
from anovos.data_transformer.transformers import imputation_sklearn

In [11]:
df = df.drop('empty')

In [12]:
print(df.count())
print(df.dropna().count())

32561
11641


In [13]:
# Example 1 - with mandatory arguments + KNN method  + print impact
odf = imputation_sklearn(spark, idf=df, print_impact=True)

[Stage 50:>                                                         (0 + 2) / 2]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3

+--------------+-------------------+------------------+
|attribute     |missingCount_before|missingCount_after|
+--------------+-------------------+------------------+
|education-num |31                 |0                 |
|capital-gain  |13                 |0                 |
|capital-loss  |12                 |0                 |
|age           |61                 |0                 |
|hours-per-week|109                |0                 |
|fnlwgt        |15                 |0                 |
|logfnl        |20393              |0                 |
+--------------+-------------------+------------------+



In [54]:
# Example 2 - selected columns + regression method + print impact
odf = imputation_sklearn(spark, idf=df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                         method_type='regression', print_impact=True)

[Stage 1377:>                                                       (0 + 1) / 1]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning:

+-------------+-------------------+------------------+
|attribute    |missingCount_before|missingCount_after|
+-------------+-------------------+------------------+
|education-num|31                 |0                 |
|capital-gain |13                 |0                 |
|capital-loss |12                 |0                 |
|age          |61                 |0                 |
+-------------+-------------------+------------------+



In [55]:
# Example 3 - KNN method + smaller sample_size + save model
odf = imputation_sklearn(spark, idf=df, sample_size=1000, model_path=outputPath+'/KNN')

[Stage 1418:>                                                       (0 + 1) / 1]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3

In [56]:
from anovos.data_analyzer.stats_generator import measures_of_percentiles, measures_of_counts
x = measures_of_counts(spark, odf)

# Visualization
x.orderBy('missing_count').toPandas() 

,attribute,fill_count,fill_pct,missing_count,missing_pct,nonzero_count,nonzero_pct
0,ifa,32561,1.0000,0,0.0000,NaN,NaN
1,education-num,32561,1.0000,0,0.0000,32561.0,1.0000
2,capital-gain,32561,1.0000,0,0.0000,2715.0,0.0834
3,capital-loss,32561,1.0000,0,0.0000,1520.0,0.0467
4,income,32561,1.0000,0,0.0000,NaN,NaN
5,age,32561,1.0000,0,0.0000,32561.0,1.0000
6,hours-per-week,32561,1.0000,0,0.0000,32561.0,1.0000
7,fnlwgt,32561,1.0000,0,0.0000,32561.0,1.0000
8,native-country,32561,1.0000,0,0.0000,NaN,NaN
9,logfnl,32561,1.0000,0,0.0000,32561.0,1.0000


In [57]:
# Example 4 - KNN method + pre-saved model + append new columns + print impact
odf = imputation_sklearn(spark, idf=df, pre_existing_model=True, model_path=outputPath+'/KNN', 
                         output_mode='append', print_impact=True)

[Stage 1454:>                                                       (0 + 1) / 1]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3

+--------------+-------------------+----------------------+-------------+
|attribute     |missingCount_before|attribute_after       |missing_count|
+--------------+-------------------+----------------------+-------------+
|capital-gain  |13                 |capital-gain_imputed  |0            |
|capital-loss  |12                 |capital-loss_imputed  |0            |
|fnlwgt        |15                 |fnlwgt_imputed        |0            |
|hours-per-week|109                |hours-per-week_imputed|0            |
|age           |61                 |age_imputed           |0            |
|education-num |31                 |education-num_imputed |0            |
|logfnl        |20393              |logfnl_imputed        |0            |
+--------------+-------------------+----------------------+-------------+



In [58]:
# Example 5 - regression method + smaller sample_size + save model
odf = imputation_sklearn(spark, idf=df, sample_size=1000, model_path=outputPath+'/regression')

/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
[Stage 1493:>                                                       (0 + 1) / 1]/Users/sinuochen/anaconda3/envs/py37/lib/python3

In [59]:
# Example 6 - regression method + pre-saved model + append new columns + print impact
odf = imputation_sklearn(spark, idf=df, pre_existing_model=True, model_path=outputPath+'/regression', 
                         output_mode='append', print_impact=True)

[Stage 1508:>                                                       (0 + 1) / 1]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3

+--------------+-------------------+----------------------+-------------+
|attribute     |missingCount_before|attribute_after       |missing_count|
+--------------+-------------------+----------------------+-------------+
|capital-gain  |13                 |capital-gain_imputed  |0            |
|capital-loss  |12                 |capital-loss_imputed  |0            |
|fnlwgt        |15                 |fnlwgt_imputed        |0            |
|hours-per-week|109                |hours-per-week_imputed|0            |
|age           |61                 |age_imputed           |0            |
|education-num |31                 |education-num_imputed |0            |
|logfnl        |20393              |logfnl_imputed        |0            |
+--------------+-------------------+----------------------+-------------+



In [14]:
# Example 7 - use pre-saved stats
from anovos.data_analyzer.stats_generator import measures_of_counts
from anovos.data_ingest.data_ingest import write_dataset

write_dataset(measures_of_counts(spark, df), outputPath+"/missing","parquet", file_configs={"mode":"overwrite"})

odf = imputation_sklearn(spark, df, stats_missing={"file_path":outputPath+"/missing", "file_type": "parquet"}, 
                         print_impact=True)

[Stage 95:>                                                         (0 + 2) / 2]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3

+--------------+-------------------+------------------+
|attribute     |missingCount_before|missingCount_after|
+--------------+-------------------+------------------+
|hours-per-week|109                |0                 |
|education-num |31                 |0                 |
|capital-gain  |13                 |0                 |
|capital-loss  |12                 |0                 |
|fnlwgt        |15                 |0                 |
|logfnl        |20393              |0                 |
|age           |61                 |0                 |
+--------------+-------------------+------------------+



## Imputation Matrix Factorization
- API specification of function **imputation_matrixFactorization** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only

In [61]:
from anovos.data_transformer.transformers import imputation_matrixFactorization

In [62]:
# Example 1 - all columns with missing values + print impact
odf = imputation_matrixFactorization(spark, idf=df, id_col='ifa', print_impact=True)

+--------------+-------------------+------------------+
|attribute     |missingCount_before|missingCount_after|
+--------------+-------------------+------------------+
|education-num |31                 |0                 |
|capital-gain  |13                 |0                 |
|capital-loss  |12                 |0                 |
|age           |61                 |0                 |
|hours-per-week|109                |0                 |
|fnlwgt        |15                 |0                 |
|logfnl        |20393              |0                 |
+--------------+-------------------+------------------+



In [63]:
# Example 2 - selected columns + append new columns + print impact
odf = imputation_matrixFactorization(spark, idf=df, 
                                     list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                                     id_col='ifa', print_impact=True)

+-------------+-------------------+------------------+
|attribute    |missingCount_before|missingCount_after|
+-------------+-------------------+------------------+
|education-num|31                 |0                 |
|capital-gain |13                 |0                 |
|capital-loss |12                 |0                 |
|age          |61                 |0                 |
+-------------+-------------------+------------------+



In [64]:
# Example 3 - use pre-saved stats
from anovos.data_analyzer.stats_generator import measures_of_counts
from anovos.data_ingest.data_ingest import write_dataset

write_dataset(measures_of_counts(spark, df), outputPath+"/missing","parquet", file_configs={"mode":"overwrite"})

odf = imputation_matrixFactorization(spark, df, 
                                     stats_missing={"file_path":outputPath+"/missing", "file_type": "parquet"}, 
                                     print_impact=True)

+--------------+-------------------+------------------+
|attribute     |missingCount_before|missingCount_after|
+--------------+-------------------+------------------+
|hours-per-week|109                |0                 |
|education-num |31                 |0                 |
|capital-gain  |13                 |0                 |
|capital-loss  |12                 |0                 |
|logfnl        |20393              |0                 |
|fnlwgt        |15                 |0                 |
|age           |61                 |0                 |
+--------------+-------------------+------------------+



## Auto Imputation
- API specification of function **auto_imputation** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>

In [67]:
from anovos.data_transformer.transformers import auto_imputation

In [66]:
# Example 1 - all columns with missing values + print impact
auto_imputation(spark, df, id_col='ifa', print_impact=True)

[Stage 2685:>                                                       (0 + 1) / 1]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
- _4
- ...
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num
- fnlwgt
- ...

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3

[('MMM-mean', 3.527763864712216), ('MMM-median', 3.92952606466687), ('KNN', 3.9519728198861728), ('regression', 3.498162349389901), ('matrix_factorization', 5.876657881935616)]
Best Imputation Method:  regression



[Stage 3784:===========================================>       (172 + 12) / 200]


DataFrame[ifa: string, age: float, fnlwgt: float, logfnl: float, education-num: float, capital-gain: float, capital-loss: float, hours-per-week: float, native-country: string, income: string, education: string, sex: string, marital-status: string, workclass: string, occupation: string, race: string, relationship: string, index: int]

In [69]:
# Example 2 - selected columns + customized null_pct + print impact
odf = auto_imputation(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'],
                                   id_col='ifa', null_pct=0.5, print_impact=True)

/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/anovos/data_transformer/transformers.py:2154: UserWarning: Following columns have all null values: empty
  warnings.warn("Following columns have all null values: " + ",".join(empty_cols))
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num

  warnings.warn(message, FutureWarning)
[Stage 802:>                                                        (0 + 1) / 1]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1

[('MMM-mean', 8.45557875153865), ('MMM-median', 16.711628363631945), ('KNN', 8.863458483329977), ('regression', 9.384259370085479), ('matrix_factorization', 11.739216950649823)]
Best Imputation Method:  MMM-mean


In [70]:
# Example 3 - selected columns + use pre-saved stats + print impact
from anovos.data_analyzer.stats_generator import measures_of_counts
from anovos.data_ingest.data_ingest import write_dataset

write_dataset(measures_of_counts(spark, df), outputPath+"/missing","parquet", file_configs={"mode":"overwrite"})

odf = auto_imputation(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                      id_col='ifa', stats_missing={"file_path":outputPath+"/missing", "file_type": "parquet"}, 
                      print_impact=True)

/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/anovos/data_transformer/transformers.py:2154: UserWarning: Following columns have all null values: empty
  warnings.warn("Following columns have all null values: " + ",".join(empty_cols))
[Stage 1555:>                                                       (0 + 1) / 1]/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1
- _2
- _3
Feature names seen at fit time, yet now missing:
- age
- capital-gain
- capital-loss
- education-num

  warnings.warn(message, FutureWarning)
/Users/sinuochen/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _0
- _1

[('MMM-mean', 3.7619303948670058), ('MMM-median', 4.2195222297059685), ('KNN', 4.037273402022951), ('regression', 3.7190839693796875), ('matrix_factorization', 4.725047562065969)]
Best Imputation Method:  regression



[Stage 2255:=============================================>     (179 + 12) / 200]


# Latent Features Generation

## Autoencoder Latent Features
- API specification of function **autoencoder_latentFeatures** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only

In [71]:
from anovos.data_transformer.transformers import autoencoder_latentFeatures

In [72]:
# Example 1 - with mandatory arguments + print impact
odf = autoencoder_latentFeatures(spark, df, print_impact=True)
odf.limit(5).toPandas()

2022-03-03 15:50:32.854344: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
38/38 - 1s - loss: 0.9551 - val_loss: 0.9484 - 1s/epoch - 38ms/step
Epoch 2/100
38/38 - 0s - loss: 0.6548 - val_loss: 0.8467 - 76ms/epoch - 2ms/step
Epoch 3/100
38/38 - 0s - loss: 0.5365 - val_loss: 0.7400 - 73ms/epoch - 2ms/step
Epoch 4/100
38/38 - 0s - loss: 0.4544 - val_loss: 0.6300 - 70ms/epoch - 2ms/step
Epoch 5/100
38/38 - 0s - loss: 0.4076 - val_loss: 0.5372 - 70ms/epoch - 2ms/step
Epoch 6/100
38/38 - 0s - loss: 0.3725 - val_loss: 0.4625 - 72ms/epoch - 2ms/step
Epoch 7/100
38/38 - 0s - loss: 0.3470 - val_loss: 0.4143 - 70ms/epoch - 2ms/step
Epoch 8/100
38/38 - 0s - loss: 0.3276 - val_loss: 0.3760 - 70ms/epoch - 2ms/step
Epoch 9/100
38/38 - 0s - loss: 0.3140 - val_loss: 0.3531 - 75ms/epoch - 2ms/step
Epoch 10/100
38/38 - 0s - loss: 0.2974 - val_loss: 0.3277 - 74ms/epoch - 2ms/step
Epoch 11/100
38/38 - 0s - loss: 0.2814 - val_loss: 0.3088 - 67ms/epoch - 2ms/step
Epoch 12/100
38/38 - 0s - loss: 0.2726 - val_loss: 0.2915 - 67ms/epoch - 2ms/step
Epoch 13/100
38/38 - 0s - 

[Stage 2290:>                                                       (0 + 1) / 1]2022-03-03 15:50:46.214497: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


+-------+-------------------+-------------------+------------------+
|summary|latent_0           |latent_1           |latent_2          |
+-------+-------------------+-------------------+------------------+
|count  |12085              |12085              |12085             |
|mean   |-0.7191699736328255|0.24448327120311827|1.013178770622753 |
|stddev |1.310171735813494  |1.094015521649406  |1.8547920443439576|
|min    |-4.2180247         |-1.8480381         |-8.207147         |
|max    |6.6346774          |9.401324           |12.226687         |
+-------+-------------------+-------------------+------------------+



[Stage 2292:>                                                       (0 + 1) / 1]2022-03-03 15:50:50.227874: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,ifa,workclass,empty,education,marital-status,occupation,relationship,race,sex,native-country,...,capital-gain_scaled,hours-per-week_scaled,education-num_scaled,logfnl_scaled,capital-loss_scaled,fnlwgt_scaled,age_scaled,latent_0,latent_1,latent_2
0,3a,Private,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,UnitedStates,...,-0.145898,-0.020960,-0.420201,0.467746,-0.216698,0.245012,-0.037494,-1.190261,-0.263054,1.010719
1,4a,Private,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,UnitedStates,...,-0.145898,-0.020960,-1.197652,0.601973,-0.216698,0.425709,1.072918,-1.373738,-0.714812,0.788100
2,6a,Private,None,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,United-States,...,-0.145898,-0.020960,1.523426,0.907015,-0.216698,0.898043,-0.111522,-0.030955,0.323042,0.829771
3,7a,Private,None,None,None,Other-service,Not-in-family,Black,Female,Jamaica,...,-0.145898,-2.035340,-1.975102,-0.003056,-0.216698,-0.280352,0.776808,-1.891652,-0.879023,0.636174
4,8a,Self-emp-not-inc,None,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,...,-0.145898,0.398703,-0.420201,0.423030,-0.216698,0.188136,0.998890,-0.824820,-0.408954,0.341645


In [73]:
# Example 2 - selected columns + less epochs + larger bach size + print impact
odf = autoencoder_latentFeatures(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'],
                                 epochs=50, batch_size=528, print_impact=True)
odf.limit(5).toPandas()

Epoch 1/50
50/50 - 1s - loss: 0.8571 - val_loss: 3.9614 - 909ms/epoch - 18ms/step
Epoch 2/50
50/50 - 0s - loss: 0.5185 - val_loss: 3.7777 - 75ms/epoch - 1ms/step
Epoch 3/50
50/50 - 0s - loss: 0.4463 - val_loss: 3.3755 - 71ms/epoch - 1ms/step
Epoch 4/50
50/50 - 0s - loss: 0.3886 - val_loss: 2.7009 - 77ms/epoch - 2ms/step
Epoch 5/50
50/50 - 0s - loss: 0.3462 - val_loss: 2.0886 - 70ms/epoch - 1ms/step
Epoch 6/50
50/50 - 0s - loss: 0.3107 - val_loss: 1.5453 - 75ms/epoch - 1ms/step
Epoch 7/50
50/50 - 0s - loss: 0.2872 - val_loss: 1.1516 - 72ms/epoch - 1ms/step
Epoch 8/50
50/50 - 0s - loss: 0.2552 - val_loss: 0.9059 - 71ms/epoch - 1ms/step
Epoch 9/50
50/50 - 0s - loss: 0.2312 - val_loss: 0.7750 - 71ms/epoch - 1ms/step
Epoch 10/50
50/50 - 0s - loss: 0.2070 - val_loss: 0.6723 - 74ms/epoch - 1ms/step
Epoch 11/50
50/50 - 0s - loss: 0.1869 - val_loss: 0.6203 - 76ms/epoch - 2ms/step
Epoch 12/50
50/50 - 0s - loss: 0.1637 - val_loss: 0.5719 - 73ms/epoch - 1ms/step
Epoch 13/50
50/50 - 0s - loss: 0.16

[Stage 2320:>                                                       (0 + 1) / 1]2022-03-03 15:51:01.274627: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


+-------+---------------------+-------------------+
|summary|latent_0             |latent_1           |
+-------+---------------------+-------------------+
|count  |32466                |32466              |
|mean   |-0.022195468554359592|0.06505244099466488|
|stddev |0.36544642955295464  |0.6886165742920519 |
|min    |-3.6273923           |-8.733038          |
|max    |2.3011553            |1.2474811          |
+-------+---------------------+-------------------+



[Stage 2322:>                                                       (0 + 1) / 1]2022-03-03 15:51:05.919812: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,ifa,workclass,fnlwgt,logfnl,empty,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income,capital-gain_scaled,capital-loss_scaled,education-num_scaled,age_scaled,latent_0,latent_1
0,3a,Private,215646,5.333741,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,UnitedStates,<=50K,-0.145898,-0.216698,-0.420201,-0.037494,-0.086085,0.178734
1,4a,Private,234721,5.370552,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,UnitedStates,<=50K,-0.145898,-0.216698,-1.197652,1.072918,-0.256777,0.515634
2,6a,Private,284582,5.454207,None,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,<=50K,-0.145898,-0.216698,1.523426,-0.111522,0.198821,0.118660
3,7a,Private,160187,5.204627,None,None,None,Other-service,Not-in-family,Black,Female,16,Jamaica,<=50K,-0.145898,-0.216698,-1.975102,0.776808,-0.433187,0.499542
4,8a,Self-emp-not-inc,209642,5.321478,None,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,-0.145898,-0.216698,-0.420201,0.998890,-0.082220,0.433774


In [74]:
# Example 3 - selected columns + smaller sample_size used for training + save model
odf = autoencoder_latentFeatures(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'],
                                 sample_size=20000, model_path=outputPath)
odf.limit(5).toPandas()

Epoch 1/100
63/63 - 1s - loss: 0.8582 - val_loss: 4.0904 - 892ms/epoch - 14ms/step
Epoch 2/100
63/63 - 0s - loss: 0.6411 - val_loss: 3.7127 - 87ms/epoch - 1ms/step
Epoch 3/100
63/63 - 0s - loss: 0.5635 - val_loss: 3.4301 - 78ms/epoch - 1ms/step
Epoch 4/100
63/63 - 0s - loss: 0.4644 - val_loss: 3.1557 - 78ms/epoch - 1ms/step
Epoch 5/100
63/63 - 0s - loss: 0.4229 - val_loss: 2.8891 - 79ms/epoch - 1ms/step
Epoch 6/100
63/63 - 0s - loss: 0.3890 - val_loss: 2.5892 - 78ms/epoch - 1ms/step
Epoch 7/100
63/63 - 0s - loss: 0.3649 - val_loss: 2.2885 - 80ms/epoch - 1ms/step
Epoch 8/100
63/63 - 0s - loss: 0.3370 - val_loss: 1.9839 - 78ms/epoch - 1ms/step
Epoch 9/100
63/63 - 0s - loss: 0.3124 - val_loss: 1.7350 - 79ms/epoch - 1ms/step
Epoch 10/100
63/63 - 0s - loss: 0.2924 - val_loss: 1.4413 - 78ms/epoch - 1ms/step
Epoch 11/100
63/63 - 0s - loss: 0.2668 - val_loss: 1.2321 - 90ms/epoch - 1ms/step
Epoch 12/100
63/63 - 0s - loss: 0.2486 - val_loss: 1.0060 - 82ms/epoch - 1ms/step
Epoch 13/100
63/63 - 0s

[Stage 2350:>                                                       (0 + 1) / 1]2022-03-03 15:51:21.444623: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,ifa,workclass,fnlwgt,logfnl,empty,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income,capital-gain_scaled,capital-loss_scaled,education-num_scaled,age_scaled,latent_0,latent_1
0,3a,Private,215646,5.333741,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,UnitedStates,<=50K,-0.145898,-0.216698,-0.420201,-0.037494,-0.469321,0.291389
1,4a,Private,234721,5.370552,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,UnitedStates,<=50K,-0.145898,-0.216698,-1.197652,1.072918,-0.613352,0.378457
2,6a,Private,284582,5.454207,None,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,<=50K,-0.145898,-0.216698,1.523426,-0.111522,-0.161768,-0.027954
3,7a,Private,160187,5.204627,None,None,None,Other-service,Not-in-family,Black,Female,16,Jamaica,<=50K,-0.145898,-0.216698,-1.975102,0.776808,-0.676020,0.460595
4,8a,Self-emp-not-inc,209642,5.321478,None,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,-0.145898,-0.216698,-0.420201,0.998890,-0.523731,0.288951


In [75]:
# Example 4 - use pre-saved model
odf = autoencoder_latentFeatures(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                                 pre_existing_model=True, model_path=outputPath, print_impact=True)

[Stage 2372:>                                                       (0 + 1) / 1]2022-03-03 15:51:27.295336: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


+-------+-------------------+-------------------+
|summary|latent_0           |latent_1           |
+-------+-------------------+-------------------+
|count  |32466              |32466              |
|mean   |-0.3213917958361229|0.16583641601817148|
|stddev |0.41658311723759656|0.3625252611643652 |
|min    |-1.0048386         |-2.9771426         |
|max    |3.3970242          |2.1176977          |
+-------+-------------------+-------------------+



In [76]:
# Example 5 - selected columns + use pre-saved stats + print impact
from anovos.data_analyzer.stats_generator import measures_of_counts
from anovos.data_ingest.data_ingest import write_dataset

write_dataset(measures_of_counts(spark, df), outputPath+"/missing","parquet", file_configs={"mode":"overwrite"})

odf = autoencoder_latentFeatures(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                                 stats_missing={"file_path":outputPath+"/missing", "file_type": "parquet"}, 
                                 print_impact=True)

Epoch 1/100
102/102 - 1s - loss: 0.6283 - val_loss: 3.0130 - 939ms/epoch - 9ms/step
Epoch 2/100
102/102 - 0s - loss: 0.4170 - val_loss: 2.2634 - 121ms/epoch - 1ms/step
Epoch 3/100
102/102 - 0s - loss: 0.3367 - val_loss: 1.3787 - 116ms/epoch - 1ms/step
Epoch 4/100
102/102 - 0s - loss: 0.2837 - val_loss: 0.7238 - 116ms/epoch - 1ms/step
Epoch 5/100
102/102 - 0s - loss: 0.2538 - val_loss: 0.4648 - 117ms/epoch - 1ms/step
Epoch 6/100
102/102 - 0s - loss: 0.2249 - val_loss: 0.4364 - 115ms/epoch - 1ms/step
Epoch 7/100
102/102 - 0s - loss: 0.2053 - val_loss: 0.3936 - 116ms/epoch - 1ms/step
Epoch 8/100
102/102 - 0s - loss: 0.1894 - val_loss: 0.3884 - 117ms/epoch - 1ms/step
Epoch 9/100
102/102 - 0s - loss: 0.1804 - val_loss: 0.4021 - 116ms/epoch - 1ms/step
Epoch 10/100
102/102 - 0s - loss: 0.1693 - val_loss: 0.3638 - 115ms/epoch - 1ms/step
Epoch 11/100
102/102 - 0s - loss: 0.1539 - val_loss: 0.3467 - 120ms/epoch - 1ms/step
Epoch 12/100
102/102 - 0s - loss: 0.1465 - val_loss: 0.3397 - 116ms/epoch 

Epoch 98/100
102/102 - 0s - loss: 0.0733 - val_loss: 0.2508 - 117ms/epoch - 1ms/step
Epoch 99/100
102/102 - 0s - loss: 0.0758 - val_loss: 0.2436 - 117ms/epoch - 1ms/step
Epoch 100/100
102/102 - 0s - loss: 0.0749 - val_loss: 0.2473 - 118ms/epoch - 1ms/step


[Stage 2406:>                                                       (0 + 1) / 1]2022-03-03 15:51:49.465450: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


+-------+-------------------+--------------------+
|summary|latent_0           |latent_1            |
+-------+-------------------+--------------------+
|count  |32466              |32466               |
|mean   |0.45503874619093293|-0.24757758470433366|
|stddev |0.7782824058443919 |0.6664926673751039  |
|min    |-2.9453254         |-6.439064           |
|max    |8.352249           |2.43653             |
+-------+-------------------+--------------------+



In [77]:
# Example 6 - use pre-saved standardization model
odf = autoencoder_latentFeatures(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                                 standardization_configs={"pre_existing_model": True, "model_path": outputPath}, 
                                 print_impact=True)


[Stage 2420:==================================================> (196 + 4) / 200]


Epoch 1/100
102/102 - 1s - loss: 0.8848 - val_loss: 2.7341 - 1s/epoch - 11ms/step
Epoch 2/100
102/102 - 0s - loss: 0.4462 - val_loss: 1.3794 - 122ms/epoch - 1ms/step
Epoch 3/100
102/102 - 0s - loss: 0.3137 - val_loss: 0.6684 - 121ms/epoch - 1ms/step
Epoch 4/100
102/102 - 0s - loss: 0.2718 - val_loss: 0.4923 - 116ms/epoch - 1ms/step
Epoch 5/100
102/102 - 0s - loss: 0.2390 - val_loss: 0.4589 - 123ms/epoch - 1ms/step
Epoch 6/100
102/102 - 0s - loss: 0.2242 - val_loss: 0.4481 - 120ms/epoch - 1ms/step
Epoch 7/100
102/102 - 0s - loss: 0.2059 - val_loss: 0.4429 - 116ms/epoch - 1ms/step
Epoch 8/100
102/102 - 0s - loss: 0.1860 - val_loss: 0.4354 - 119ms/epoch - 1ms/step
Epoch 9/100
102/102 - 0s - loss: 0.1702 - val_loss: 0.4165 - 116ms/epoch - 1ms/step
Epoch 10/100
102/102 - 0s - loss: 0.1547 - val_loss: 0.4163 - 117ms/epoch - 1ms/step
Epoch 11/100
102/102 - 0s - loss: 0.1436 - val_loss: 0.4057 - 120ms/epoch - 1ms/step
Epoch 12/100
102/102 - 0s - loss: 0.1418 - val_loss: 0.4043 - 118ms/epoch - 

Epoch 98/100
102/102 - 0s - loss: 0.0444 - val_loss: 0.2099 - 121ms/epoch - 1ms/step
Epoch 99/100
102/102 - 0s - loss: 0.0453 - val_loss: 0.2061 - 119ms/epoch - 1ms/step
Epoch 100/100
102/102 - 0s - loss: 0.0426 - val_loss: 0.2049 - 118ms/epoch - 1ms/step


[Stage 2432:>                                                       (0 + 1) / 1]2022-03-03 15:52:10.905690: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


+-------+-------------------+--------------------+
|summary|latent_0           |latent_1            |
+-------+-------------------+--------------------+
|count  |32466              |32466               |
|mean   |0.15136716302809802|-0.39043422079285833|
|stddev |1.3492937426394687 |0.7153738594675946  |
|min    |-3.8490484         |-7.114113           |
|max    |14.527969          |3.95306             |
+-------+-------------------+--------------------+



In [78]:
# Example 7 - impute missing values before calculation
odf = autoencoder_latentFeatures(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                                 imputation=True, print_impact=True)


[Stage 2446:==================================================> (195 + 5) / 200]


Epoch 1/100
102/102 - 1s - loss: 1.4239 - val_loss: 3.7064 - 1s/epoch - 13ms/step
Epoch 2/100
102/102 - 0s - loss: 0.7415 - val_loss: 3.2492 - 149ms/epoch - 1ms/step
Epoch 3/100
102/102 - 0s - loss: 0.6449 - val_loss: 2.8163 - 153ms/epoch - 2ms/step
Epoch 4/100
102/102 - 0s - loss: 0.6026 - val_loss: 2.5033 - 150ms/epoch - 1ms/step
Epoch 5/100
102/102 - 0s - loss: 0.5702 - val_loss: 2.2419 - 153ms/epoch - 2ms/step
Epoch 6/100
102/102 - 0s - loss: 0.5345 - val_loss: 1.9488 - 157ms/epoch - 2ms/step
Epoch 7/100
102/102 - 0s - loss: 0.4360 - val_loss: 1.2304 - 146ms/epoch - 1ms/step
Epoch 8/100
102/102 - 0s - loss: 0.3756 - val_loss: 0.7509 - 160ms/epoch - 2ms/step
Epoch 9/100
102/102 - 0s - loss: 0.3460 - val_loss: 0.6448 - 152ms/epoch - 1ms/step
Epoch 10/100
102/102 - 0s - loss: 0.3330 - val_loss: 0.6224 - 157ms/epoch - 2ms/step
Epoch 11/100
102/102 - 0s - loss: 0.3040 - val_loss: 0.6229 - 157ms/epoch - 2ms/step
Epoch 12/100
102/102 - 0s - loss: 0.2903 - val_loss: 0.6091 - 152ms/epoch - 

Epoch 98/100
102/102 - 0s - loss: 0.0762 - val_loss: 0.3835 - 148ms/epoch - 1ms/step
Epoch 99/100
102/102 - 0s - loss: 0.0723 - val_loss: 0.3710 - 153ms/epoch - 2ms/step
Epoch 100/100
102/102 - 0s - loss: 0.0692 - val_loss: 0.3660 - 151ms/epoch - 1ms/step


[Stage 2469:>                                                       (0 + 1) / 1]2022-03-03 15:52:36.573911: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


+-------+-------------------+------------------+
|summary|latent_0           |latent_1          |
+-------+-------------------+------------------+
|count  |32561              |32561             |
|mean   |0.817135767840456  |0.8327797176445377|
|stddev |0.39537309884884064|1.4572281826150844|
|min    |0.17767985         |-13.167922        |
|max    |4.5946302          |4.390725          |
+-------+-------------------+------------------+



## PCA Latent Features
- API specification of function **PCA_latentFeatures** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only

In [79]:
from anovos.data_transformer.transformers import PCA_latentFeatures

In [80]:
# Example 1 - with mandatory arguments + print impact
odf = PCA_latentFeatures(spark, df, standardization=True, print_impact=True)
odf.limit(5).toPandas()


[Stage 2483:===================================================>(199 + 1) / 200]


Explained Variance:  0.9866



[Stage 2508:>                                                       (0 + 1) / 1]


+-------+--------------------+---------------------+-------------------+---------------------+--------------------+---------------------+
|summary|latent_0            |latent_1             |latent_2           |latent_3             |latent_4            |latent_5             |
+-------+--------------------+---------------------+-------------------+---------------------+--------------------+---------------------+
|count  |12085               |12085                |12085              |12085                |12085               |12085                |
|mean   |0.006990473625180995|0.0016755667182939693|0.00531931809024164|-0.009834625663188677|0.007587318127886196|-0.008249612368146014|
|stddev |1.381797632041396   |1.1228193008371192   |1.015036593603689  |0.9838047392914232   |0.9419796818084403  |0.8971511772848009   |
|min    |-3.6521552          |-3.5124547           |-8.272692          |-2.186532            |-7.3917856          |-7.4833627           |
|max    |9.021795            |8.73

,ifa,workclass,empty,education,marital-status,occupation,relationship,race,sex,native-country,income,latent_0,latent_1,latent_2,latent_3,latent_4,latent_5
0,3a,Private,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,UnitedStates,<=50K,0.544680,-0.323292,-0.042623,0.146981,0.248483,-0.113231
1,4a,Private,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,UnitedStates,<=50K,0.707140,-0.398122,0.210681,1.422995,0.590206,-0.326371
2,6a,Private,None,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,United-States,<=50K,1.169402,0.817424,-0.208944,-0.570670,-0.319456,1.234487
3,7a,Private,None,None,None,Other-service,Not-in-family,Black,Female,Jamaica,<=50K,-0.014027,-2.157362,0.256786,1.788715,-0.760998,-0.507208
4,8a,Self-emp-not-inc,None,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,>50K,0.342476,0.209419,0.112829,0.979675,0.671754,0.071360


In [81]:
# Example 2 - selected columns + customized explained_variance_cutoff + print impact
odf = PCA_latentFeatures(spark, df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                         explained_variance_cutoff=0.6, standardization=True, print_impact=True)
odf.limit(5).toPandas()

Explained Variance:  0.7943


+-------+---------------------+----------------------+---------------------+
|summary|latent_0             |latent_1              |latent_2             |
+-------+---------------------+----------------------+---------------------+
|count  |32466                |32466                 |32466                |
|mean   |1.6552674503466662E-4|-6.3589239401116544E-6|2.8812043641422073E-4|
|stddev |1.0870763482758605   |1.0143077110112224    |0.9826621520658779   |
|min    |-9.827796            |-9.095775             |-2.3905334           |
|max    |2.9694445            |7.914514              |4.3574286            |
+-------+---------------------+----------------------+---------------------+



,ifa,workclass,fnlwgt,logfnl,empty,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income,latent_0,latent_1,latent_2
0,3a,Private,215646,5.333741,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,UnitedStates,<=50K,0.423203,0.102860,0.229906
1,4a,Private,234721,5.370552,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,UnitedStates,<=50K,0.380385,-0.019731,1.574839
2,6a,Private,284582,5.454207,None,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,<=50K,-0.725312,0.091507,-0.904990
3,7a,Private,160187,5.204627,None,None,None,Other-service,Not-in-family,Black,Female,16,Jamaica,<=50K,0.991146,0.023124,1.760285
4,8a,Self-emp-not-inc,209642,5.321478,None,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,-0.058382,-0.019048,1.084266


In [82]:
# Example 3 - selected columns + save model
odf = PCA_latentFeatures(spark, df, model_path=outputPath, standardization=True)


[Stage 2557:==============================================>    (181 + 12) / 200]


In [83]:
# Example 4 - selected columns + use pre-saved model
odf = PCA_latentFeatures(spark, df, pre_existing_model=True, model_path=outputPath, standardization=True, 
                         print_impact=True)
odf.limit(5).toPandas()

Explained Variance:  0.9866
+-------+--------------------+---------------------+-------------------+---------------------+--------------------+---------------------+
|summary|latent_0            |latent_1             |latent_2           |latent_3             |latent_4            |latent_5             |
+-------+--------------------+---------------------+-------------------+---------------------+--------------------+---------------------+
|count  |12085               |12085                |12085              |12085                |12085               |12085                |
|mean   |0.006990473625180995|0.0016755667182939693|0.00531931809024164|-0.009834625663188677|0.007587318127886196|-0.008249612368146014|
|stddev |1.381797632041396   |1.1228193008371192   |1.015036593603689  |0.9838047392914232   |0.9419796818084403  |0.8971511772848009   |
|min    |-3.6521552          |-3.5124547           |-8.272692          |-2.186532            |-7.3917856          |-7.4833627           |
|max  

,ifa,workclass,empty,education,marital-status,occupation,relationship,race,sex,native-country,income,latent_0,latent_1,latent_2,latent_3,latent_4,latent_5
0,3a,Private,None,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,UnitedStates,<=50K,0.544680,-0.323292,-0.042623,0.146981,0.248483,-0.113231
1,4a,Private,None,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,UnitedStates,<=50K,0.707140,-0.398122,0.210681,1.422995,0.590206,-0.326371
2,6a,Private,None,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,United-States,<=50K,1.169402,0.817424,-0.208944,-0.570670,-0.319456,1.234487
3,7a,Private,None,None,None,Other-service,Not-in-family,Black,Female,Jamaica,<=50K,-0.014027,-2.157362,0.256786,1.788715,-0.760998,-0.507208
4,8a,Self-emp-not-inc,None,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,>50K,0.342476,0.209419,0.112829,0.979675,0.671754,0.071360


In [84]:
# Example 5 - selected columns + use pre-saved stats + print impact
from anovos.data_analyzer.stats_generator import measures_of_counts
from anovos.data_ingest.data_ingest import write_dataset

write_dataset(measures_of_counts(spark, df), outputPath+"/missing","parquet", file_configs={"mode":"overwrite"})

odf = PCA_latentFeatures(spark, df, standardization=True, 
                         stats_missing={"file_path":outputPath+"/missing", "file_type": "parquet"}, 
                         print_impact=True)

Explained Variance:  0.9866
+-------+--------------------+---------------------+-------------------+---------------------+--------------------+---------------------+
|summary|latent_0            |latent_1             |latent_2           |latent_3             |latent_4            |latent_5             |
+-------+--------------------+---------------------+-------------------+---------------------+--------------------+---------------------+
|count  |12085               |12085                |12085              |12085                |12085               |12085                |
|mean   |0.006990473625180995|0.0016755667182939693|0.00531931809024164|-0.009834625663188677|0.007587318127886196|-0.008249612368146014|
|stddev |1.381797632041396   |1.1228193008371192   |1.015036593603689  |0.9838047392914232   |0.9419796818084403  |0.8971511772848009   |
|min    |-3.6521552          |-3.5124547           |-8.272692          |-2.186532            |-7.3917856          |-7.4833627           |
|max  

In [85]:
# Example 6 - use pre-saved standardization model
odf = PCA_latentFeatures(spark, df, standardization=True,
                         standardization_configs={"pre_existing_model": True, "model_path": outputPath}, 
                         print_impact=True)

Explained Variance:  0.9866
+-------+--------------------+---------------------+-------------------+---------------------+--------------------+---------------------+
|summary|latent_0            |latent_1             |latent_2           |latent_3             |latent_4            |latent_5             |
+-------+--------------------+---------------------+-------------------+---------------------+--------------------+---------------------+
|count  |12085               |12085                |12085              |12085                |12085               |12085                |
|mean   |0.006990473625180995|0.0016755667182939693|0.00531931809024164|-0.009834625663188677|0.007587318127886196|-0.008249612368146014|
|stddev |1.381797632041396   |1.1228193008371192   |1.015036593603689  |0.9838047392914232   |0.9419796818084403  |0.8971511772848009   |
|min    |-3.6521552          |-3.5124547           |-8.272692          |-2.186532            |-7.3917856          |-7.4833627           |
|max  

In [86]:
# Example 7 - impute missing values before calculation
odf = PCA_latentFeatures(spark, df, standardization=True, imputation=True, print_impact=True)

Explained Variance:  0.9635



[Stage 2744:>                                                       (0 + 1) / 1]

+-------+--------------------+--------------------+-------------------+--------------------+---------------------+---------------------+
|summary|latent_0            |latent_1            |latent_2           |latent_3            |latent_4             |latent_5             |
+-------+--------------------+--------------------+-------------------+--------------------+---------------------+---------------------+
|count  |32561               |32561               |32561              |32561               |32561                |32561                |
|mean   |0.014842900705817235|0.037540262676611896|0.00452105304795438|0.010756455649333153|0.0016000300296498654|-0.006853182355680893|
|stddev |1.1508209468691626  |1.0680661788123114  |1.0138911021934351 |0.9808178415429736  |0.938791376448427    |0.8967908005138374   |
|min    |-8.268953           |-3.943358           |-7.9534187         |-2.5194323          |-7.6249423           |-3.4757562           |
|max    |5.2560782           |9.891342   

# Feature Transformation
- API specification of function **feature_transformation** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only

In [87]:
from anovos.data_transformer.transformers import feature_transformation

In [88]:
# Example 1: sqrt 
odf = feature_transformation(idf=df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                             method_type='sqrt', print_impact=True)

Before:
+-------+------------------+-----------------+------------------+------------------+
|summary|capital-gain      |capital-loss     |education-num     |age               |
+-------+------------------+-----------------+------------------+------------------+
|count  |32548             |32549            |32530             |32500             |
|mean   |1077.6959567408135|87.3360164674798 |10.080971411005226|38.506492307692305|
|stddev |7386.624857802765 |403.0310072565718|2.5725103263986977|13.508497735339255|
|min    |0                 |0                |1                 |17                |
|max    |99999             |4356             |16                |85                |
+-------+------------------+-----------------+------------------+------------------+

After:
+-------+------------------+------------------+------------------+------------------+
|summary|capital-gain      |capital-loss      |education-num     |age               |
+-------+------------------+------------------+

In [89]:
# Example 2: log + append generated columns
odf = feature_transformation(idf=df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                             method_type='ln', output_mode='append', print_impact=True)

Before:
+-------+------------------+-----------------+------------------+------------------+
|summary|capital-gain      |capital-loss     |education-num     |age               |
+-------+------------------+-----------------+------------------+------------------+
|count  |32548             |32549            |32530             |32500             |
|mean   |1077.6959567408135|87.3360164674798 |10.080971411005226|38.506492307692305|
|stddev |7386.624857802765 |403.0310072565718|2.5725103263986977|13.508497735339255|
|min    |0                 |0                |1                 |17                |
|max    |99999             |4356             |16                |85                |
+-------+------------------+-----------------+------------------+------------------+

After:
+-------+------------------+-------------------+------------------+-------------------+
|summary|capital-gain_ln   |capital-loss_ln    |education-num_ln  |age_ln             |
+-------+------------------+---------------

In [90]:
# Example 3: round to 1 decimal place
odf = feature_transformation(idf=odf, 
                             list_of_cols=['education-num_ln', 'capital-gain_ln', 'capital-loss_ln', 'age_ln'], 
                             method_type='roundN', N=1, print_impact=True)

Before:
+-------+------------------+-------------------+-------------------+------------------+
|summary|education-num_ln  |age_ln             |capital-loss_ln    |capital-gain_ln   |
+-------+------------------+-------------------+-------------------+------------------+
|count  |32530             |32500              |1519               |2710              |
|mean   |2.2689316480632096|3.588027147805643  |7.508497766226061  |8.819883472603545 |
|stddev |0.3168442727686073|0.35895718528658876|0.25675668323690815|1.0158964531089267|
|min    |0.0               |2.833213344056216  |5.043425116919247  |4.736198448394496 |
|max    |2.772588722239781 |4.442651256490317  |8.37930948405285   |11.512915464920228|
+-------+------------------+-------------------+-------------------+------------------+

After:
+-------+-------------------+-------------------+-------------------+------------------+
|summary|education-num_ln   |age_ln             |capital-loss_ln    |capital-gain_ln   |
+-------+-----

In [91]:
# Example 4: square
odf = feature_transformation(idf=df, list_of_cols='age', method_type='sq', print_impact=True)

Before:
+-------+------------------+
|summary|age               |
+-------+------------------+
|count  |32500             |
|mean   |38.506492307692305|
|stddev |13.508497735339255|
|min    |17                |
|max    |85                |
+-------+------------------+

After:
+-------+------------------+
|summary|age               |
+-------+------------------+
|count  |32500             |
|mean   |1665.2238461538461|
|stddev |1154.2085383349045|
|min    |289.0             |
|max    |7225.0            |
+-------+------------------+



In [92]:
# Example 5: remainder divided by 10
odf = feature_transformation(idf=df, list_of_cols=['education-num', 'capital-gain', 'capital-loss', 'age'], 
                             method_type='remainderDivByN', N=10, print_impact=True)

Before:
+-------+------------------+-----------------+------------------+------------------+
|summary|capital-gain      |capital-loss     |education-num     |age               |
+-------+------------------+-----------------+------------------+------------------+
|count  |32548             |32549            |32530             |32500             |
|mean   |1077.6959567408135|87.3360164674798 |10.080971411005226|38.506492307692305|
|stddev |7386.624857802765 |403.0310072565718|2.5725103263986977|13.508497735339255|
|min    |0                 |0                |1                 |17                |
|max    |99999             |4356             |16                |85                |
+-------+------------------+-----------------+------------------+------------------+

After:
+-------+-------------------+------------------+------------------+------------------+
|summary|capital-gain       |capital-loss      |education-num     |age               |
+-------+-------------------+----------------

# Box Cox Transformation
- API specification of function **boxcox_transformation** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports numerical attributes only

In [93]:
from anovos.data_transformer.transformers import boxcox_transformation

In [94]:
# Example 1 - selected columns + print impact
odf = boxcox_transformation(df, drop_cols=['capital-loss', 'capital-gain'], print_impact=True)

Transformed Columns:  ['education-num', 'hours-per-week', 'logfnl', 'fnlwgt', 'age']
Best BoxCox Parameter(s):  [3, 3, 1, 1, 0]
Before:
+--------+-------------------+--------------------+-------------------+------------------+------------------+
|summary |education-num      |hours-per-week      |logfnl             |fnlwgt            |age               |
+--------+-------------------+--------------------+-------------------+------------------+------------------+
|count   |32530              |32452               |12168              |32546             |32500             |
|mean    |10.080971411005226 |40.24972266732405   |5.2054654851899365 |189781.83180728814|38.506492307692305|
|stddev  |2.5725103263986977 |11.914337669272212  |0.27424241727170395|105563.06445057027|13.508497735339255|
|min     |1                  |1                   |4.283617786        |12285             |17                |
|max     |16                 |94                  |6.088696941        |1484705           |85  

In [95]:
# Example 2 - selected columns + existing lambda value + print impact
odf = boxcox_transformation(df, list_of_cols='age', boxcox_lambda=0, output_mode='append', print_impact=True)

Transformed Columns:  ['age']
Best BoxCox Parameter(s):  [0]
Before:
+--------+------------------+
|summary |age               |
+--------+------------------+
|count   |32500             |
|mean    |38.506492307692305|
|stddev  |13.508497735339255|
|min     |17                |
|max     |85                |
|skewness|0.5127993362812463|
+--------+------------------+

After:
+--------+--------------------+
|summary |age_bxcx_0          |
+--------+--------------------+
|count   |32500               |
|mean    |3.588027147805643   |
|stddev  |0.35895718528658876 |
|min     |2.833213344056216   |
|max     |4.442651256490317   |
|skewness|-0.14607838263666595|
+--------+--------------------+



# Outlier Categories Treatment
- API specification of function **outlier_categories** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>
- Supports 2 ways of outliers detection: by max number of categories and by coverage (%)

In [16]:
from anovos.data_transformer.transformers import outlier_categories

In [17]:
# Example 1 - 'all' columns (excluding drop_cols) + max 15 categories + append transformed columns at the end
odf = outlier_categories(spark, df, drop_cols=['ifa'], max_category=15, output_mode='append')
odf.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,...,occupation_outliered,empty_outliered,relationship_outliered,marital-status_outliered,workclass_outliered,education_outliered,sex_outliered,native-country_outliered,race_outliered,income_outliered
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,...,Adm-clerical,None,Not-in-family,Never-married,State-gov,Bachelors,Male,outlier_categories,White,<=50K
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,...,Exec-managerial,None,Husband,Married-civ-spouse,Self-emp-not-inc,Bachelors,Male,outlier_categories,White,<=50K
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,...,Handlers-cleaners,None,Not-in-family,Divorced,Private,HS-grad,Male,outlier_categories,White,<=50K
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,...,Handlers-cleaners,None,Husband,Married-civ-spouse,Private,11th,Male,outlier_categories,Black,<=50K
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,...,Prof-specialty,None,Wife,Married-civ-spouse,Private,Bachelors,Female,Cuba,Black,<=50K


In [18]:
# Example 2 - selected columns + max 10 categories
odf = outlier_categories(spark, df, list_of_cols=['education', 'occupation', 'native-country'], 
                         max_category=10, print_impact=True)

+--------------+-------------------+
|attribute     |uniqueValues_before|
+--------------+-------------------+
|occupation    |15                 |
|education     |16                 |
|native-country|44                 |
+--------------+-------------------+

+--------------+------------------+
|attribute     |uniqueValues_after|
+--------------+------------------+
|occupation    |10                |
|education     |10                |
|native-country|10                |
+--------------+------------------+



In [19]:
# Example 3 - selected columns + cover 90% values
odf = outlier_categories(spark, df, list_of_cols=['education', 'occupation', 'native-country'], 
                         coverage=0.9, print_impact=True)

+--------------+-------------------+
|attribute     |uniqueValues_before|
+--------------+-------------------+
|occupation    |15                 |
|education     |16                 |
|native-country|44                 |
+--------------+-------------------+

+--------------+------------------+
|attribute     |uniqueValues_after|
+--------------+------------------+
|occupation    |11                |
|education     |9                 |
|native-country|3                 |
+--------------+------------------+



In [20]:
# Example 4 - max 15 categories + save model
odf = outlier_categories(spark, df, drop_cols=['ifa'], max_category=15, 
                         pre_existing_model=False, model_path=outputPath, print_impact=True)

+--------------+-------------------+
|attribute     |uniqueValues_before|
+--------------+-------------------+
|occupation    |15                 |
|relationship  |8                  |
|empty         |0                  |
|marital-status|7                  |
|workclass     |11                 |
|education     |16                 |
|sex           |3                  |
|native-country|44                 |
|race          |9                  |
|income        |2                  |
+--------------+-------------------+

+--------------+------------------+
|attribute     |uniqueValues_after|
+--------------+------------------+
|occupation    |15                |
|relationship  |8                 |
|empty         |0                 |
|marital-status|7                 |
|workclass     |11                |
|education     |15                |
|sex           |3                 |
|native-country|15                |
|race          |9                 |
|income        |2                 |
+------------

In [21]:
# Example 5 - use pre-saved model
odf = outlier_categories(spark, df, drop_cols=['ifa'], pre_existing_model=True, model_path=outputPath, print_impact=True)

+--------------+-------------------+
|attribute     |uniqueValues_before|
+--------------+-------------------+
|occupation    |15                 |
|relationship  |8                  |
|empty         |0                  |
|marital-status|7                  |
|workclass     |11                 |
|education     |16                 |
|sex           |3                  |
|native-country|44                 |
|race          |9                  |
|income        |2                  |
+--------------+-------------------+

+--------------+------------------+
|attribute     |uniqueValues_after|
+--------------+------------------+
|occupation    |15                |
|relationship  |8                 |
|empty         |0                 |
|marital-status|7                 |
|workclass     |10                |
|education     |15                |
|sex           |3                 |
|native-country|15                |
|race          |9                 |
|income        |2                 |
+------------

# Expression Parser
- API specification of function **expression_parser** can be found <a href="https://docs.anovos.ai/api/data_transformer/transformers.html">here</a>

In [102]:
from anovos.data_transformer.transformers import expression_parser

In [103]:
# Example 1 - 2 generated columns + print impact
odf = expression_parser(df, ['age + hours-per-week', 'capital-gain-capital-loss'], print_impact=True)

Columns Added:  ['f0', 'f1']
+-------+------------------+-----------------+
|summary|f0                |f1               |
+-------+------------------+-----------------+
|count  |32392             |32548            |
|mean   |78.75373549024451 |990.3572569743149|
|stddev |18.619824518135385|7410.325259409019|
|min    |20                |-4356            |
|max    |158               |99999            |
+-------+------------------+-----------------+



In [104]:
# Example 1 - 2 generated columns + print impact
odf = expression_parser(df, ['age + hours-per-week', 'capital-gain/capital-loss'], print_impact=True)

Columns Added:  ['f0', 'f1']
+-------+------------------+----+
|summary|f0                |f1  |
+-------+------------------+----+
|count  |32392             |1519|
|mean   |78.75373549024451 |0.0 |
|stddev |18.619824518135385|0.0 |
|min    |20                |0.0 |
|max    |158               |0.0 |
+-------+------------------+----+



In [105]:
# Example 2 - 2 generated columns + customized postfix + print impact
odf = expression_parser(df, ['age + hours-per-week', 'capital-gain - capital-loss'], postfix="_new", print_impact=True)

Columns Added:  ['f0_new', 'f1_new']
+-------+------------------+-----------------+
|summary|f0_new            |f1_new           |
+-------+------------------+-----------------+
|count  |32392             |32548            |
|mean   |78.75373549024451 |990.3572569743149|
|stddev |18.619824518135385|7410.325259409019|
|min    |20                |-4356            |
|max    |158               |99999            |
+-------+------------------+-----------------+

